# Resampler Demo

Authors: David Atkinson

First version: 20 June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).
Copyright 2021 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

In [ ]:
# Setup the working directory for the notebook
# (Requires download_data.sh to have been run once before)
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('Geometry')

In [ ]:
# Import required packages
import nibabel 
import matplotlib.pyplot as plt
import numpy as np
import os

import sirf.Reg as Reg

In [ ]:
# Data for geometry notebooks when run is ./nifti/*.nii
data_path = os.getcwd()

This demo uses the resampler in SIRF to regrid one scan into the geometry of another. A phantom was scanned with slices in the coronal orientation and again with slices in an oblique orientation, without moving the phantom. 
In this Notebook, we read and display the NIfTI images using `nibabel`, then read using SIRF and demonstrate a resampling of one into the geometry of the other.

In [ ]:
# Set the full file names for the data files
fpath  = os.path.join(data_path , 'nifti')
fn_cor = "OBJECT_phantom_T2W_TSE_Cor_14_1.nii" # Coronal volume, 30 slices
fn_obl = "OBJECT_phantom_T2W_TSE_obl_19_1.nii"

ffn_cor = os.path.join(fpath, fn_cor)  # full file name
print("Full file name: ", ffn_cor)

ffn_obl = os.path.join(fpath, fn_obl)  # full file name
print("Full file name: ", ffn_obl)

In [ ]:
# Read in NIfTIs using nibabel and get the image data
n_cor = nibabel.load(ffn_cor)
n_obl = nibabel.load(ffn_obl)

nad_cor = n_cor.get_fdata()  # NIfTI image array data
nad_obl = n_obl.get_fdata()  # NIfTI image array data

In [ ]:
# Display a slice from each dataset
slc = 15 ;

slc_obl = nad_obl[:,:,slc]
plt.subplot(1,2,1, title='Oblique')
plt.imshow(slc_obl)

slc_cor = nad_cor[:,:,slc]
plt.subplot(1,2,2, title='Coronal')
plt.imshow(slc_cor)


Note distortion is visible in the Oblique scan.

In [ ]:
# Now read in using SIRF
R_obl = Reg.ImageData(ffn_obl)
R_cor = Reg.ImageData(ffn_cor)

In [ ]:
# Construct an identity transformation object
# We need an identity because we are using the resampler only to resample
# and not to apply any additional transformation. 

tm_identity = np.array([[1,0,0,0],
                        [0,1,0,0],
                        [0,0,1,0],
                        [0,0,0,1]])

TM = Reg.AffineTransformation(tm_identity)

In [ ]:
# Adapted from Richard Brown's notebook: sirf_registration

# Create a resampler and set the reference image to the Coronal object
resampler = Reg.NiftyResample()
resampler.set_reference_image(R_cor)

# Set the floating image to be resampled to the Oblique
resampler.set_floating_image(R_obl)

# Add the desired transformation (here identity)
resampler.add_transformation(TM)
resampler.set_padding_value(0)
# Use linear interpolation
resampler.set_interpolation_type_to_linear()
# Go!
resampler.process()

plt.figure()
plt.subplot(1,2,1, title='Oblique resampled to Coronal')
plt.imshow(resampler.get_output().as_array()[:,:,slc])
plt.subplot(1,2,2, title='Original Coronal')
plt.imshow(slc_cor)


The images are not identical - so is this correct? The regions where there are missing sections are OK because that part of the data was never in the oblique scan. The rectangular region in the circular phantom is a little different. It is hard to tell if this is a small error or because the distortions were different in the scans. (Distortions can be seen in the previous images).
The stripes in the resampled image could be an interpolation issue.


# Reading with sirf.STIR

`sirf.STIR` can also read NIfTI files (and DICOM files if STIR was compiled with ITK support). STIR internally uses a gantry-based coordinate system and STIR will re-orient images such that the first dimension corresponds to axial slices. See example below and warnings that when using STIR, data should not be oblique and only for patients with a head-first-supine orientation. 

Let's see how reading with `sirf.STIR` works out here.

In [ ]:
# Most exercises import `sirf.STIR` as `pet`, but here we call it differently as it's used to read MR data...
import sirf.STIR as STIR

In [ ]:
S_cor = STIR.ImageData(ffn_cor)  # The coronal image as a sirf.STIR object

In [ ]:
# STIR re-sorted the data in a different order
print(S_cor.as_array().shape)
print(R_cor.as_array().shape)  # The coronal image read above as a sirf.Reg object

In [ ]:
print(S_cor.get_geometrical_info().get_info())
print(R_cor.get_geometrical_info().get_info())

In [ ]:
# However, it kept track of geometry, so resampling should still work
resampler.set_reference_image(R_cor)

# Set the "STIR" image as the floating image that will be resampled 
resampler.set_floating_image(S_cor)
resampler.process()

plt.figure()
plt.subplot(1,2,1, title='Coronal read via sirf.STIR')
plt.imshow(resampler.get_output().as_array()[:,:,slc])
plt.subplot(1,2,2, title='Original Coronal (read using nibabel')
plt.imshow(slc_cor)


In [ ]:
print(resampler.get_output().get_geometrical_info().get_info())
print(R_cor.get_geometrical_info().get_info())

***WARNING***. At present, STIR cannot handle oblique slices. However, it does not complain and silently returns incorrect geometric information. Doing the above with the *oblique* scan will therefore give an **incorrect** result (with STIR 4.*).

 **WARNING**. To be able to re-orient to gantry coordinates, STIR needs to know the patient position. This is available in DICOM data, but unfortunately not for NIfTI. STIR will therefore **assume that NIfTI images correspond to data acquired in *Head First Supine (HFS)* patient position**, as this is the most common case. This doesn't matter as far as geometry goes (i.e. LPS coordinates will be correct), but does matter when using NIfTI images as input for simulation and reconstruction (as it would "position" the patient wrongly).